In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.functions import current_timestamp, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

pit_stops_schema =  StructType(fields=[
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), True),
    StructField("stop", IntegerType(), True),
    StructField("lap", StringType(), True),
    StructField("time",StringType(), True),
    StructField("duration", StringType(), True),
    StructField("milliseconds", IntegerType(), True),
    ])

In [0]:
pit_stops_df = spark.read.schema(pit_stops_schema)\
    .option('multiLine', True)\
    .json(f'{raw_folder_path}/{v_file_date}/pit_stops.json')

In [0]:
final_df = pit_stops_df.withColumnRenamed('raceId', 'race_id')\
    .withColumnRenamed('driverId', 'driver_id')\
    .withColumn('ingestion_date', current_timestamp())\
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(v_file_date))


In [0]:

merge_condition = "tgt.race_id = src.race_id AND tgt.driver_id = src.driver_id AND tgt.stop = src.stop AND tgt.race_id = src.race_id"
merge_delta_data(final_df,'f1_processed', 'pit_stops', processed_folder_path, merge_condition, 'race_id')

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql
select * from f1_processed.pit_stops